In [ ]:
import os
import json
import re
from pathlib import Path

# Set your base directory path
base_dir = Path('Patches')

# Function to parse icon names from file names
def parse_icon_name(filename):
    name_part = filename.split('FID_')[-1]
    name_part = name_part.split('.esp')[0].split('.esm')[0]
    return name_part.strip().replace('_', ' ')

# Collect icons
data = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.lower().endswith('.svg'):
            relative_path = os.path.join(root, file).replace('\\', '/').replace(' ', '%20')
            name = parse_icon_name(file)
            
            # Extract folder name from path using regex
            match = re.search(r'Distinct%20(.*?)%20Patch', relative_path, re.IGNORECASE)
            parent_folder = match.group(1).replace('%20', ' ').strip() if match else 'Unknown'

            data.append({
                "path": relative_path,
                "name": name,
                "folder": parent_folder
            })

data.sort(key=lambda x: (x['folder'].lower()))

# Save icons.json
with open('icons.json', 'w') as f:
    json.dump(data, f, indent=4)

# Load template.html
with open('template.html', 'r', encoding='utf-8') as f:
    template_html = f.read()

# Generate HTML entries
icon_entries = ""
current_folder = None

for icon in data:
    
    if icon['folder'] != current_folder:
        if current_folder is not None:
            icon_entries += '</div>'  # Close previous folder row div
        current_folder = icon['folder']

        icon_entries += f'<h2 class="mt-5 mb-3">{current_folder}</h2><div class="row g-4">'

    icon_entries += (
        f'<div class="col-6 col-md-3">'
        f'<div class="icon-card">'
        f'<img src="{icon["path"]}" alt="{icon["name"]}">'
        f'<div class="icon-name">{icon["name"]}</div>'
        f'</div></div>'
    )

if current_folder is not None:
    icon_entries += '</div>'  # Close the last folder row div


# Insert entries into template
index_html = template_html.replace('<!-- implement here -->', icon_entries)

# Save index.html
with open('index.html', 'w', encoding='utf-8') as f:
    f.write(index_html)

print(f"Generated icons.json and index.html from template with {len(data)} entries.")


NameError: name 'current_folder' is not defined